In [14]:
using Plots
#using Pkg; Pkg.add("DistributedArrays")
#using Transducers
using FileIO
using Distributed
using DistributedArrays
using DistributedArrays.SPMD
#@everywhere using SharedArrays
@everywhere using Distributed
@everywhere using DistributedArrays
@everywhere using DistributedArrays.SPMD
using BenchmarkTools

In [2]:
const N_workers = 4
addprocs(N_workers)

4-element Vector{Int64}:
 2
 3
 4
 5

In [3]:
nprocs()
procs()

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [10]:
#rmprocs(6,7,8,9)
workers()
#nworkers()

4-element Vector{Int64}:
 2
 3
 4
 5

In [4]:
@everywhere include_string(Main, $(read("aux_functions.jl", String)), "aux_functions.jl")

In [5]:
order = 1
const L = 5
#N = 80000
const N = 20000
const J = 50
const κ = 2π/L # for Fourier Transform
dx = L/J
x = [dx*(i-1) for i in 1:J] ;
p = (L, N, J, κ, dx, order)

(5, 20000, 50, 1.2566370614359172, 0.1, 1)

In [11]:
d_closeall()


In [15]:
#DS = dzeros((J,4), workers(), [1,4])
DS = dzeros((J,nworkers()), workers(), [1,nworkers()]);
#pars = dzeros((N), workers(), [nworkers()] )

50×4 DArray{Float64, 2, Matrix{Float64}}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 ⋮              
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [16]:
#pars = dzeros((20), workers()[1:nprocs()-1], [nprocs()-1])
pars = zeros(2N)
#pars = dones((N,), workers(), [nworkers()] );
par_dis = load("Initial_Distributions/par_dis_L5_N2_5_theta001.jld2", "par_dis");
reorder_particles!(par_dis,pars)

In [17]:
Dpars = distribute(pars);

In [63]:
@everywhere function impri(u,S)
    println(myid(), localindices(S), localindices(u))
    println(S[:L])
end

spmd(impri, pars, S)

1(1:0, 1:0)(1:0,)
Matrix{Float64}(undef, 0, 0)
      From worker 3:	3(1:50, 2:2)(5001:10000,)
      From worker 3:	[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]
      From worker 5:	5(1:50, 4:4)(15001:20000,)
      From worker 4:	4(1:50, 3:3)(10001:15000,)
      From worker 5:	[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]
      From worker 4:	[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 

In [86]:
@everywhere function trys(u,S)
    ran_1, ran_2 = localindices(S)
    LS = S[:L]
        for j in ran_1
        LS[j,:] .= j*myid()
        end
    #return S[:,:]
end

    

In [87]:
spmd(trys, pars, S; pids=workers()) 

In [91]:
S;

In [90]:
Sf = zeros(J)
for j in 1:J
    for i in 1:nworkers()
        Sf[j] += S[j,i]
    end
end
 

In [44]:
@everywhere function get_current_ro_range(Dpars::DArray, DS::DArray, p)
    L, N, J, κ, dx, order = p
    #LN = floor(Int,N ÷ nworkers())
    Lu = Dpars[:L]
    @show ran_S1, ran_S2 = localindices(DS)
    @show ran_pars = localindices(Dpars)
    #assuming ran_pars[1] is odd
    #ran_init, ran_end = ran_pars
    @show odd_range = first(ran_pars):2:last(ran_pars)
    #even_range = (ran_init + 1):2:ran_end
    LS = DS[:L]
    #r = view(Lu,odd_range)
    #v = view(Lu,even_range)
    fill!(LS,0.0)
    #for i in odd_range
      #@show(range)
      #j, y = get_index_and_y(Lu[i],J,L)
      #for l in (-order):order 
        #LS[mod1(j + l, J),:] += W(order, -y + l) * Lu[i+1] / dx;
      #end
    #end
  end

(ran_S1, ran_S2) = localindices(DS) = (1:0, 1:0)
ran_pars = localindices(Dpars) = (1:0,)
      From worker 4:	(ran_S1, ran_S2) = localindices(DS) = (1:50, 3:3)
      From worker 3:	(ran_S1, ran_S2) = localindices(DS) = (1:50, 2:2)
      From worker 3:	ran_pars = localindices(Dpars) = (10001:20000,)
      From worker 4:	ran_pars = localindices(Dpars) = (20001:30000,)
      From worker 2:	(ran_S1, ran_S2) = localindices(DS) = (1:50, 1:1)
      From worker 2:	ran_pars = localindices(Dpars) = (1:10000,)
      From worker 5:	(ran_S1, ran_S2) = localindices(DS) = (1:50, 4:4)
      From worker 5:	ran_pars = localindices(Dpars) = (30001:40000,)


In [45]:
spmd(get_current_ro_range, Dpars, DS, p)

CompositeException: TaskFailedException

    nested task error: MethodError: no method matching +(::UnitRange{Int64}, ::Int64)
    For element-wise addition, use broadcasting with dot syntax: array .+ scalar
    Closest candidates are:
      +(::Any, ::Any, !Matched::Any, !Matched::Any...) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/operators.jl:655
      +(!Matched::T, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/int.jl:87
      +(::OrdinalRange, !Matched::OrdinalRange) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/range.jl:1347
      ...
    Stacktrace:
      [1] _colon(start::UnitRange{Int64}, step::Int64, stop::UnitRange{Int64})
        @ Base ./range.jl:45
      [2] (::Colon)(start::UnitRange{Int64}, step::Int64, stop::UnitRange{Int64})
        @ Base ./range.jl:40
      [3] macro expansion
        @ ./sh

In [111]:
ran = 3:4
ran_i, ran_e = ran
ran_e

4

In [47]:
ran = 5:10
last(ran)
first(ran)
first(ran):2:last(ran)

5:2:9